In [1]:
import os
import ast
import json
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

# —— Parameters —— #
INPUT_CSV    = '/app/src/module_3/cachedLaws_1.csv'    # your source CSV
QUERY_FILE   = '/app/src/module_4/input/query.txt'           # your query text
OUTPUT_CSV   = '/app/src/module_4/output/filtered_laws.csv'  # where to save results
THRESHOLD    = 0.5                         # similarity cutoff (0–1)

# —— Prepare output directory —— #
os.makedirs(os.path.dirname(OUTPUT_CSV), exist_ok=True)

# —— Load data —— #
# Read first 5 rows (tab-separated)
df = pd.read_csv(INPUT_CSV).head(5)
df.to_csv("input/sample_input.csv", index=False)

# Load the user query
with open(QUERY_FILE, 'r', encoding='utf-8') as f:
    query = f.read().strip()

# —— Initialize embedder —— #
model = SentenceTransformer('jinaai/jina-embeddings-v2-small-en')
query_emb = model.encode(query, convert_to_numpy=True)

# —— Processing function —— #
def filter_structured(sj):
    """
    Parse the Python-style JSON in sj, compute cosine similarity of each
    article/annex text to the query, add 'score' and keep only elements
    with score >= THRESHOLD.
    """
    # Parse Python-dict string
    try:
        data = ast.literal_eval(sj)
    except (ValueError, SyntaxError):
        return {'articles': [], 'annexes': []}

    out = {'articles': [], 'annexes': []}
    for section in ('articles', 'annexes'):
        for elem in data.get(section, []):
            text = elem.get('text', '')
            # embed and compute cosine similarity
            emb = model.encode(text, convert_to_numpy=True)
            score = float(
                np.dot(query_emb, emb) /
                (np.linalg.norm(query_emb) * np.linalg.norm(emb))
            )
            elem['score'] = score
            if score >= THRESHOLD:
                out[section].append(elem)
    return out

# —— Apply filtering —— #
df['filtered_json'] = df['structured_json'].apply(filter_structured)

# Keep only rows with at least one match
mask = df['filtered_json'].apply(lambda d: len(d['articles']) + len(d['annexes']) > 0)
df_filtered = df[mask].copy()

# Convert filtered dict back to JSON string
df_filtered['filtered_json'] = df_filtered['filtered_json'].apply(json.dumps)

# Select relevant columns and save
result = df_filtered[['celex_id', 'reference', 'summary', 'filtered_json']]
result.to_csv(OUTPUT_CSV, index=False)


/usr/local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-small-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.2.intermediate.dense.bias', 'encoder.layer.2.intermediate.dense.w

In [5]:
result.to_json('/app/src/module_4/output/filtered_laws.json', orient='records', lines=True)